**Step 1: Find all patients with condition X from three sources**

**Required**
* Connect to FHIR Synthetic Patient Data
* Connect to OHDSI Synthetic Patient Data
* Connect to HUSH+ Synthetic Patient Data

**Step 1.1: Find 

In [3]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator

DISEASE2SYMPTOMS = [x.split("\t") for x in urlopen("https://www.nature.com/article-assets/npg/ncomms/2014/140626/ncomms5212/extref/ncomms5212-s4.txt").read().split("\n")]
DISEASE2SYMPTOMS = filter(lambda x: len(x) == 4, DISEASE2SYMPTOMS)

In [2]:
#connect to UMLS
try:
    cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print
    #cnx.close()

In [3]:
#Define functions
def findICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        return (icd10[0])
    return ("Undef")


In [4]:
asthmaCodes = findICD10('Asthma')
findICD10('Asthma')

u'J45.9'

In [6]:
## FHIR patient data
import requests
import urllib2
import json

#base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
#patients = base_url+"/Patient"+"?_count=50"

#req = requests.get(patients)

def findPatients(code):
    try:
        response = urllib2.urlopen("http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+code+"")
    except:
        print "I can't find patients"
        exit()
    html = response.read()
    return (html)

In [7]:
## Find FHIR patients with asthma codes
for c in asthmaCodes:
    print findPatients(c)


	{
    "resourceType" : "Bundle",
    "id" : "7B5A5E7B-5056-8036-4FCC48C734624219",
    "meta" : {
        "count" : "50",
        "lastUpdated" : "2017-05-10T10:21:27Z"
    },
    "type" : "collection",
    "entry" : [
	
		{
			"fullurl" : "http://ictrweb.johnshopkins.edu/rest/synthetic/Condition/1654",
				"resourceType" : "Condition", 
				"id": "1654",
				"meta": {
				  "versionId": "2",
				  "lastUpdated": "2017-05-10T10:21:27Z"
				},
				"status": "final",
				"clinicalStatus": "active",
				"verificationStatus": "confirmed",
				"category": [
				{
				  "coding": [
					{
					  "system": "http://snomed.info/sct",
					  "code": "55607006",
					  "display": "Problem"
					},
					{
					  "system": "http://hl7.org/fhir/condition-category",
					  "code": "problem-list-item"
					}
				  ]
				}
				],
				"severity": {
				"coding": [
				  {
					"system": "http://snomed.info/sct",
					"code": "371924009",
					"display": "Moderate to severe"
				  }
				]
				},
				"c

In [ ]:
## Given disease/condition term, get back ICD codes
def txt2icd(txt):
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": "Asthma",
              "VOCABULARY_ID": ["ICD9CM"]
    }
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(response.text.decode('utf-8'))
    icd9arr=[]
    for d in data:
        icd9arr.append(d["CONCEPT_CODE"])
    return icd9arr
print txt2icd('asthma')

In [45]:
## Given disease name, get back symptoms (defined using MeSH terms) along with TFIDF scores
## Taken from https://www.nature.com/articles/ncomms5212
def disease2symptom(txt):
    s = filter(lambda x: txt.lower() in x[1].lower(), DISEASE2SYMPTOMS)
    return([(x[0], x[3]) for x in s])
disease = "Asthma"
symps = disease2symptom(disease)
print 'Found %s symptom MeSH terms for %s' % (len(symps), disease)

Found 206 symptom MeSH terms for Asthma


In [4]:
## Pull patients matching a list of ICD codes
query = GreenTranslator ().get_query ()

patients = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
##pprint.pprint (patients)

Initializing thread provenance @thread: MainThread


In [6]:
meds = {}
for x in patients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)

{None: 9,
 u'0.3 ML Epinephrine 1 MG/ML Auto-Injector [Epipen]': 1,
 u'120 ACTUAT Fluticasone propionate 0.05 MG/ACTUAT Nasal Inhaler': 2,
 u'24 HR Methylphenidate Hydrochloride 10 MG Extended Release Oral Capsule [Ritalin]': 1,
 u'ACYCLOVIR 5 % TOPICAL OINTMENT': 1,
 u'ADVAIR HFA 230-21 MCG INHALER': 1,
 u'ALBUTEROL 90 MCG INHALER': 1,
 u'ALBUTEROL SULF HFA 90 MCG INH': 7,
 u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 9,
 u'ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER': 19,
 u'AMOXICILLIN 250 MG CHEWABLE TABLET': 1,
 u'AMOXICILLIN 875 MG TABLET': 1,
 u'AUGMENTIN 400-57 MG/5 ML': 1,
 u'AUGMENTIN ES-600 SUSPENSION': 3,
 u'AZITHROMYCIN 250 MG TABLET': 11,
 u'Acetaminophen 21.7 MG/ML / Hydrocodone Bitartrate 0.5 MG/ML Oral Solution': 1,
 u'Acetaminophen 32 MG/ML Oral Suspension': 1,
 u'Albuterol 0.83 MG/ML Inhalant Solution': 3,
 u'Amitriptyline Hydrochloride 10 MG Oral Tablet': 1,
 u'Amoxicillin 120 MG/ML / Clavulanate 8.58 MG/ML Oral Suspension': 4,
 u

In [7]:
## For a given medication string get NCBO annotations
## We let NCBO match any ontology since just using RxNORM doesn't
## always give us just the drug name (e.g., "CLINDAMYCIN 15 MG/ML ORAL SOLUTION" is
## a valid RxNORM term)
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    txt = urlopen(url % quote_plus(txt)).read()
    resp = json.loads(txt)
    if len(resp) == 0: return([])
    annos = []
    for aresp in resp:
        annos.extend([ x['text'] for x in aresp['annotations'] ])
    ##annos = filter(lambda x: not any(d in x for d in'0123456789'), annos)
    return(annos)
print med2rxnorm("CLINDAMYCIN 15 MG/ML ORAL SOLUTION")        


[u'CLINDAMYCIN 15 MG/ML ORAL SOLUTION', u'CLINDAMYCIN 15 MG/ML', u'CLINDAMYCIN', u'ORAL SOLUTION']


In [ ]:
## Get RxNORM codes for medication strings
medrxnorm = {}
for med in meds.keys():
    if med is None: continue    
    annos = med2rxnorm(med)
    print 'Processing %s and found %d annotations' % (med, len(annos))
    medrxnorm[med] = {'count':meds[med], 'annos':annos}    